# Задание **Lite**

Преобразуйте в одну функцию распознавание речи с микрофона при помощи Google API. Функция должна возвращать распознанный текст.

# Установка и подключение библиотек

## Установка

In [16]:
# !pip install SpeechRecognition
# !pip install ffmpeg-python
!pip install jiwer 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from jiwer import wer

## Подключение

In [2]:
import speech_recognition as sR
import jiwer as wer
import ffmpeg
from IPython.display import HTML, Audio
from google.colab import files
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import scipy
import librosa
import os
import IPython.display as ipd

# Создание функции

In [4]:
def Speech_to_text():

  # HTML код из занятия

  AUDIO_HTML = """
    <script>  // создаем тег <script>, сообщающий браузеру о том, что внутри находится исполняемый код JavaScript
    var my_div = document.createElement("DIV");  // создаем новый элемент DIV(тег-контейнер для логического выделения блока документа)
    var my_p = document.createElement("P");  // создаем новый элемент P(параграф для логической группировки текста)
    var my_btn = document.createElement("BUTTON");  // создаем новый элемент(кнопку) BUTTON
    var t = document.createTextNode("Press to start recording");  // создаем текстовое содержимое для кнопки

    my_btn.appendChild(t);  // добавляем текстовое содержимое элементу BUTTON
    my_div.appendChild(my_btn);  // кнопку с текстом BUTTON добавляем в блок DIV
    document.body.appendChild(my_div);  // добавляем наш блок в элемент <body> ("тело", для хранения содержимого веб-страницы)

    var base64data = 0;  // будем использовать для аудиоданных метод кодирования информации в 64-разрядный код
    var reader;  // создаем переменную для чтения файла
    var recorder, gumStream; // объявляем переменные для записи данных/потока
    var recordButton = my_btn; //  создаем переменную для кнопки записи аудио с микрофона

    var handleSuccess = function(stream) {  // объявляем функцию для работы с потоками данных
      gumStream = stream;  // создаем переменную для потока
      var options = {
        mimeType : 'audio/webm;codecs=opus' // в опциях задаем медиа тип с аудиоформатом и кодеками
      };            
      recorder = new MediaRecorder(stream); // создаем новый объект MediaRecorder, получающий медиапоток для записи.
      // MediaRecorder - интерфейс MediaStream Recording API, представляющий функциональность для простой записи медиа. Создается
      // с использованием MediaRecorder() конструктора.
      recorder.ondataavailable = function(e) {  // вызываем обработчик dataavailable события, запускаемое по окончанию записи          
        var url = URL.createObjectURL(e.data); // этим методом создаем DOMString(UTF-16 String), содержащий URL с указанием на объект e.data
        var preview = document.createElement('audio'); // создаем элемент-тег аудио
        preview.controls = true; // активизируем элементы управления
        preview.src = url; // берем в качестве исходных данных файл, содержащийся в записанной ранее URL
        document.body.appendChild(preview); // добавляем элемент аудио в <body> ("тело", для хранения содержимого веб-страницы)

        reader = new FileReader();  // создаем объект класса FileReader для чтения разных источников данных
        reader.readAsDataURL(e.data);  // читаем содержимое указанного файла
        reader.onloadend = function() {  // обработчик события, запускаемого после передачи данных
          base64data = reader.result; // записываем прочитанное содержимое в base64data
        }
      };
      recorder.start();  // начало записи медиа
      };

    recordButton.innerText = "Recording... press to stop"; // такой текст будет на кнопке BUTTON во время записи аудио

    // запрос разрешения пользователя на доступ к устройству захвата аудио (микрофон), указываем True
    navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


    function toggleRecording() {  // функция опишет действия по завершению записи (после клика мышкой по кнопке "Recording... press to stop")
      if (recorder && recorder.state == "recording") {  // если рекордер находится в процессе записи 
          recorder.stop();  // рекордер прерывается
          gumStream.getAudioTracks()[0].stop();  // отключается запись и доступ к микрофону
          recordButton.innerText = "Saving the recording... pls wait!" // эта надпись (сохранение записи) отобразится на кнопке BUTTON 
      }
    }

    // https://stackoverflow.com/a/951057
    function sleep(ms) {  // создаем функцию с задержкой вызова
      return new Promise(resolve => setTimeout(resolve, ms));  
      // new Promise - конструкция для отложенных вычислений
      // setTimeout позволяет вызвать функцию один раз через определённый интервал времени
    }

    var data = new Promise(resolve=>{
    recordButton.onclick = ()=>{  // при нажатии левой кнопкой мыши на кнопку "Recording... press to stop"
    toggleRecording()  // вызывается функция завершения аудиозаписи

    sleep(2000).then(() => {  // и после задержки 2000мс (2 сек)
      resolve(base64data.toString())  // полученные данные из формата base64 преобразовываем в строку

    });

    }
    });
          
    </script>
    """

  # Функция распознавания аудио с помощью speech_recognition
  def recognizeAudio(filename, duration=None):
      AUDIO_FILE = os.path.join(filename)
      r = sR.Recognizer()
      with sR.AudioFile(AUDIO_FILE) as source:
        audio = r.record(source, duration=duration)

      return r.recognize_google(audio, language='ru')

  # Функция извлечения аудио
  def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])

    process = (ffmpeg
      .input('pipe:0')
      .output('pipe:1', format='wav')
      .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)) 
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8 
    q = riff_chunk_size
    b = [] 
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    return audio, sr
    
  audio, sr = get_audio()
  scipy.io.wavfile.write('rec.wav', sr, audio)  # Для сохранения
  res = recognizeAudio('rec.wav', duration=None)
  return res

# Проверки

## **1** - Простое

In [20]:
original = 'проверка номер один'
res = Speech_to_text()

print('Оригинал:               ', original)
print('Результат распознавания:', res)
print('WER:', wer(original.lower(), res.lower()))

Оригинал:                проверка номер один
Результат распознавания: проверка номер один
WER: 0.0


## **2** - Сложное

### Плохое произношение 

In [21]:
original = '''С точки зрения банальной эрудиции каждый индивидуум критически 
              мотивирующий абстракцию не может игнорировать критерии утопического 
              субъективизма концептуально интерпретируя общепринятые дифференцирующие 
              поляризаторы поэтому консенсус достигнут и диалектической материальной 
              классификации всеобщих мотивации в парадигматические связи в предикатов 
              решает проблемы усовершенствования формирующих geotrans мутационных 
              возле после статов всех кинетических контролирующих аспектов'''

res = Speech_to_text()

print('Оригинал:               ', original)
print('Результат распознавания:', res)
print('WER:', wer(original.lower(), res.lower()))

Оригинал:                С точки зpения банальной эpудиции каждый индивидуум, 
              кpитически мотивиpующий абстpакцию, 
              не может игноpиpовать кpитеpии утопического субьективизма, 
              концептуально интеpпpетиpуя общепpинятые дефанизиpующие поляpизатоpы, 
              поэтому консенсус, достигнутый диалектической матеpиальной 
              классификацией всеобщих мотиваций в паpадогматических связях пpедикатов, 
              pешает пpоблему усовеpшенствования фоpмиpующих геотpансплантационных 
              квазипузлистатов всех кинетически коpеллиpующих аспектов.
Результат распознавания: С точки зрения банальной эрудиции каждый индивидуум критически мотивирующий абстракцию не может игнорировать критерии утопического субъективизма концептуально интерпретируя общепринятые деформирующий поляризаторы поэтому консенсус достигнутый диэлектрической материальной классификации всеобщих мотиваций парадигматические связи предикатов решает проблему совершенство

## Профессиональное произношение

In [32]:
original = '''С точки зрения банальной эрудиции каждый индивидуум критически мотивирующий 
              абстракцию не может игнорировать критерии утопического субъективизма 
              концептуально интерпретируя общепринятые поляризаторы поэтому консенсус 
              достигнут и диалектический материализм классификации всеобщих мотивации 
              в парадигматические связи в предикатов решая проблемы усовершенствования 
              формирующих не Отрастут акционных позиций туристов всех кинетических 
              энергий этих аспектов'''
              
res = Speech_to_text()

print('Оригинал:               ', original)
print('Результат распознавания:', res)
print('WER:', wer(original.lower(), res.lower()))

Оригинал:                С точки зрения банальной эрудиции каждый индивидуум критически мотивирующий 
              абстракцию не может игнорировать критерии утопического субъективизма 
              концептуально интерпретируя общепринятые поляризаторы поэтому консенсус 
              достигнут и диалектический материализм классификации всеобщих мотивации 
              в парадигматические связи в предикатов решая проблемы усовершенствования 
              формирующих не Отрастут акционных позиций туристов всех кинетических 
              энергий этих аспектов
Результат распознавания: С точки зрения банальной эрудиции каждый индивидуум критически мотивирующий абстракцию не может игнорировать своей теории утопического субъективизма концептуально интерпретируя общепринятые поляризаторы поэтому консенсус достигнут и диалектический материализм классификации всеобщих мотивации в парадигматические связи в предикатов решая проблемы усовершенствования формирующих делают растут акционных позиц

## **3** - Песня

In [22]:
original = '''Выйду ночью в поле с конем,
              Ночкой темной тихо пойдем,
              Мы пойдем с конем по полю вдвоем,
              Мы пойдем с конем по полю вдвоем...

              Ночью в поле звезд благодать,
              В поле никого не видать,
              Только мы с конем по полю идем,
              Только мы с конем по полю идем...'''
              
res = Speech_to_text()

print('Оригинал:               ', original)
print('Результат распознавания:', res)
print('WER:', wer(original.lower(), res.lower()))

Оригинал:                Выйду ночью в поле с конем,
              Ночкой темной тихо пойдем,
              Мы пойдем с конем по полю вдвоем,
              Мы пойдем с конем по полю вдвоем...

              Ночью в поле звезд благодать,
              В поле никого не видать,
              Только мы с конем по полю идем,
              Только мы с конем по полю идем...
Результат распознавания: песня девочка ветер тихо пойдём Мы пойдём с конём по полю пойдём
WER: 0.8958333333333334


# Выводы:

1. Использование функций сокращает код от повторений
2. В данном случаи логика работы достаточно большая и логично вынести все в отдельную функцию.
3. Как видно из примеров с простыми словами справляется намного лучше чем с трудными и длинными словами.
4. Так же результат зависит от качества произношения, дикции, интонации и т.д.
5. Из песни где тянуться последние звуки слова, так же распознается хуже, чем четко поставленные.